# Atom feed of tweeted new preprints by subject area

Preprints published in the past seven days tweeted at least three times as atom feed, grouped by subject area.

In [4]:
import sys
sys.path.append('../')
!{sys.executable} -m pip install lxml
import ast
import pandas as pd
from datetime import datetime, date
from feedgen.feed import FeedGenerator
import lxml

  Using cached lxml-4.6.4-cp38-cp38-macosx_10_14_x86_64.whl (4.5 MB)


In [5]:
date = str(date.today().strftime('%Y-%m-%d'))
subject_areas = ['covid', 'medical_health-sciences', 'bioinformatics', 'biochemistry_cell-biology', 'cancer', 'ecology', 'evolutionary-biology', 'genetics', 'microbiology', 'neurosciences', 'physiology', 'plant-biology', 'zoology','other-biological-sciences']
preprints = pd.read_csv('../preprint_tweets/1101/preprint_tweets_' + date + '.csv', index_col=0)

# filter by section, and group all COVID-related preprints together
subsets = {}
subsets['covid'] = preprints[preprints['covid'] == True]
print('COVID: ' + str(len(subsets['covid'])))
subsets['medical_health-sciences'] = preprints[((preprints['archive'] == 'medRxiv')  | (preprints['subject-area'] == 'Pathology') | (preprints['subject-area'] == 'Pharmacology and Toxicology'))  & (preprints['covid'] == False)]
print('Medical and Health Sciences: ' + str(len(subsets['medical_health-sciences'])))
subsets['bioinformatics'] = preprints[(preprints['subject-area'] == 'Bioinformatics') & (preprints['covid'] == False)]
print('Bioinformatics: ' + str(len(subsets['bioinformatics'])))
subsets['biochemistry_cell-biology'] = preprints[((preprints['subject-area'] == 'Biochemistry') | (preprints['subject-area'] == 'Cell Biology') | (preprints['subject-area'] == 'Developmental Biology') | (preprints['subject-area'] == 'Synthetic Biology') | (preprints['subject-area'] == 'Systems Biology') | (preprints['subject-area'] == 'Molecular Biology')) & (preprints['covid'] == False)]
print('Biochemistry and Cell Biology: ' + str(len(subsets['biochemistry_cell-biology'])))
subsets['ecology'] = preprints[(preprints['subject-area'] == 'Ecology') & (preprints['covid'] == False)]
subsets['cancer'] = preprints[((preprints['subject-area'] == 'Cancer Biology') | (preprints['subject-area'] == 'Hematology') | (preprints['subject-area'] == 'Oncology')) & (preprints['covid'] == False)]
print('Cancer: ' + str(len(subsets['cancer'])))
print('Ecology: ' + str(len(subsets['ecology'])))
subsets['evolutionary-biology'] = preprints[(preprints['subject-area'] == 'Evolutionary Biology') & (preprints['covid'] == False)]
print('Evolutionary Biology: ' + str(len(subsets['evolutionary-biology'])))
subsets['genetics'] = preprints[(preprints['subject-area'] == 'Genetics') | (preprints['subject-area'] == 'Genomics') & (preprints['covid'] == False)]
print('Genetics: ' + str(len(subsets['genetics'])))
subsets['microbiology'] = preprints[((preprints['subject-area'] == 'Microbiology') | (preprints['subject-area'] == 'Immunology')) & (preprints['covid'] == False)]
print('Microbiology: ' + str(len(subsets['microbiology'])))
subsets['neurosciences'] = preprints[(preprints['subject-area'] == 'Neuroscience') & (preprints['covid'] == False)]
print('Neurosciences: ' + str(len(subsets['neurosciences'])))
subsets['physiology'] = preprints[(preprints['subject-area'] == 'Physiology') | (preprints['subject-area'] == 'Biophysics') & (preprints['covid'] == False)]
print('Physiology: ' + str(len(subsets['physiology'])))
subsets['plant-biology'] = preprints[(preprints['subject-area'] == 'Plant Biology') & (preprints['covid'] == False)]
print('Plant Biology: ' + str(len(subsets['plant-biology'])))
subsets['zoology'] = preprints[(preprints['subject-area'] == 'Zoology') | (preprints['subject-area'] == 'Animal Behavior and Cognition') & (preprints['covid'] == False)]
print('Zoology: ' + str(len(subsets['zoology'])))
subsets['other-biological-sciences'] = preprints[(preprints['archive'] != 'medRxiv') & (preprints['subject-area'] != 'Animal Behavior and Cognition') & (preprints['subject-area'] != 'Bioinformatics') & (preprints['subject-area'] != 'Biochemistry') & (preprints['subject-area'] != 'Biophysics') & (preprints['subject-area'] != 'Cancer Biology') & (preprints['subject-area'] != 'Cell Biology') & (preprints['subject-area'] != 'Developmental Biology') & (preprints['subject-area'] != 'Ecology') & (preprints['subject-area'] != 'Evolutionary Biology') & (preprints['subject-area'] != 'Genetics') & (preprints['subject-area'] != 'Genomics') & (preprints['subject-area'] != 'Immunology') & (preprints['subject-area'] != 'Microbiology') & (preprints['subject-area'] != 'Molecular Biology') & (preprints['subject-area'] != 'Neuroscience') & (preprints['subject-area'] != 'Physiology') & (preprints['subject-area'] != 'Plant Biology') & (preprints['subject-area'] != 'Synthetic Biology') & (preprints['subject-area'] != 'Systems Biology') & (preprints['subject-area'] != 'Zoology') & (preprints['covid'] == False)]
print('Other Biological Sciences: ' + str(len(subsets['other-biological-sciences'])))
subsets['microbiology'].head(50)

COVID: 18
Medical and Health Sciences: 10
Bioinformatics: 10
Biochemistry and Cell Biology: 34
Cancer: 10
Ecology: 4
Evolutionary Biology: 8
Genetics: 22
Microbiology: 38
Neurosciences: 31
Physiology: 8
Plant Biology: 4
Zoology: 2
Other Biological Sciences: 9


,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
10,10.1101/2021.11.12.468384,53,bioRxiv,Microbiology,False,Phage-encoded sigma factors alter bacterial do...,"[{'name': 'Daniel A Schwartz'}, {'name': 'Bren...",<p>By entering a reversible state of reduced m...,2021-11-12
31,10.1101/2021.11.06.467345,23,bioRxiv,Microbiology,False,Recurrent urinary tract infection and estrogen...,"[{'name': 'Michael L. Neugent'}, {'name': 'Ash...",<p>Community-acquired urinary tract infection ...,2021-11-08
32,10.1101/2021.11.10.468108,22,bioRxiv,Microbiology,False,tRNA anticodon cleavage by target-activated CR...,"[{'name': 'Ishita Jain'}, {'name': 'Matvey Kol...",<p>Type VI CRISPR-Cas systems are the only CRI...,2021-11-10
38,10.1101/2021.11.08.467819,19,bioRxiv,Microbiology,False,Spatial relationships of intra-lesion heteroge...,"[{'name': 'Richard C. Lavin'}, {'name': 'Shumi...",<p>A hallmark of <italic>Mycobacterium tubercu...,2021-11-09
42,10.1101/2021.11.10.467495,17,bioRxiv,Immunology,False,Regulatory T cells suppress the formation of s...,"[{'name': 'Oksana Tsyklauri'}, {'name': 'Terez...",<p>Regulatory T cells (Tregs) are indispensabl...,2021-11-12
43,10.1101/2021.11.14.468512,17,bioRxiv,Microbiology,False,Aerobic Bacteria Produce Nitric Oxide via Deni...,"[{'name': 'Adi Abada'}, {'name': 'Martin Sperf...",<p>Microbial interactions govern marine biogeo...,2021-11-14
49,10.1101/2021.11.11.468272,15,bioRxiv,Immunology,False,PHGDH is required for germinal center formatio...,"[{'name': ""Annalisa D'Avola""}, {'name': 'Natha...",<p>The synthesis of serine from glucose is a k...,2021-11-12
53,10.1101/2021.11.09.467952,14,bioRxiv,Microbiology,False,Salmonella Outer Membrane Vesicles contain tRN...,"[{'name': 'Dominika Houserova'}, {'name': 'Yul...",<p><italic>Salmonella</italic> Outer Membrane ...,2021-11-09
55,10.1101/2021.11.08.467742,12,bioRxiv,Microbiology,False,The bacterial actin-like cell division protein...,"[{'name': 'Tim Nierhaus'}, {'name': 'Stephen H...",<p>Cell growth and division of walled bacteria...,2021-11-08
59,10.1101/2021.11.11.467955,12,bioRxiv,Microbiology,False,Linking extreme seasonality and gene expressio...,"[{'name': 'Magdalena Wutkowska'}, {'name': 'An...","<p>At high latitudes, strong seasonal differen...",2021-11-12


In [6]:

for subject_area in subject_areas:
    fg = FeedGenerator()
    fg.id('https://front-matter.io/')
    if subject_area == 'covid':
        fg.title(f'Popular BioRxiv/medRxiv COVID preprints posted in the last week')
    else:
        fg.title(f'Popular {subject_area.replace("-", " ").replace("_", " and ")} preprints posted in the last week')
    fg.author( {'name':'Martin Fenner','email':'martin@front-matter.io'} )
    fg.link( href='https://front-matter.io', rel='alternate' )
    fg.link( href=f'https://front-matter.io/{subject_area}/feed.xml', rel='self' )
    fg.language('en')

    for index, row in subsets[subject_area].iterrows():
        fe = fg.add_entry(order='append')
        fe.title(row['title'] + f" ({row['tweets']} tweets)")
        fe.author(ast.literal_eval(row['authors']))
        fe.id('https://doi.org/' + row['doi'])
        fe.link({'href': 'https://doi.org/' + row['doi'], 'title': row['title'] + f" ({row['tweets']} tweets)"})
        fe.description(row['abstract'])
        fe.category({ 'term': row['subject-area']})
        fe.published(datetime.fromisoformat(row['posted'] + 'T00:00:00+00:00'))
                 
    fg.atom_file(f'{subject_area}/feed.xml')